In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
data = pd.read_csv('tweet_emotions2.csv',encoding="latin")

# Split the data into text and labels
text = data['content'].astype(str).values
labels = data['sentiment'].values

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(text)
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

label_mapping = {'empty':0, 'sadness':1, 'enthusiasm':2, 'neutral':3, 'worry':4, 'surprise':5,
       'love':6, 'fun':7, 'hate':8, 'happiness':9, 'boredom':10, 'relief':11, 'anger':12,
       'joy':13, 'fear':14, 'shame':15, 'disgust':16}
labels = np.array([label_mapping[label] for label in labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(SimpleRNN(128))
model.add(Dense(18, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')


print("Accuracy: {:.2f}".format(accuracy))
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-Score: {:.2f}".format(f1))


Epoch 1/5
1870/1870 [==============================] - 212s 113ms/step - loss: 2.2225 - accuracy: 0.2666 - val_loss: 2.0488 - val_accuracy: 0.3321
Epoch 2/5
1870/1870 [==============================] - 203s 109ms/step - loss: 1.5874 - accuracy: 0.4954 - val_loss: 2.0720 - val_accuracy: 0.3584
Epoch 3/5
1870/1870 [==============================] - 202s 108ms/step - loss: 0.9664 - accuracy: 0.7057 - val_loss: 2.4664 - val_accuracy: 0.3415
Epoch 4/5
1870/1870 [==============================] - 206s 110ms/step - loss: 0.7081 - accuracy: 0.7867 - val_loss: 2.8807 - val_accuracy: 0.3269
Epoch 5/5
468/468 [==============================] - 7s 15ms/step
Accuracy: 0.31
Precision: 0.30
Recall: 0.31
F1-Score: 0.30
